<a href="https://colab.research.google.com/github/Ardaolmez/Sms_Classification/blob/main/Sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep="\t", header=None, names=['y', 'x'])
df_train.head()



In [ ]:
df_test = pd.read_csv(test_file_path, sep="\t", header=None, names=['y', 'x'])
df_test.head()

In [ ]:
y_train = df_train['y'].astype('category').cat.codes
y_test  = df_test['y'].astype('category').cat.codes

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopwords_eng = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_txt(txt):
    txt = re.sub(r'([^\s\w])+', ' ', txt)
    txt = " ".join([lemmatizer.lemmatize(word) for word in txt.split()
                    if not word in stopwords_eng])
    txt = txt.lower()
    return txt


In [ ]:
X_train = df_train['x'].apply(lambda x: clean_txt(x))
X_train[:5]
X_test = df_test['x'].apply(lambda x: clean_txt(x))

In [ ]:
import torch
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from collections import Counter

from collections import Counter
from torch.nn.utils.rnn import pad_sequence
import torch

# Define the tokenizer
tokenizer = get_tokenizer('basic_english')

# Keep top 1000 frequently occurring words
max_words = 1000
max_len = 500


# Tokenize the training data
tokenized_text = [tokenizer(text) for text in X_train]

# Count the word frequencies
word_freq = Counter()
for tokens in tokenized_text:
    word_freq.update(tokens)

# Sort the words by frequency in descending order
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Keep only the top 1000 words
top_words = [word for word, _ in sorted_words[:max_words]]

# Convert tokens to indices using the top words
word2idx = {word: idx for idx, word in enumerate(top_words)}
print(len(word2idx))
# Convert each tokenized text to indices
sequences = []
for tokens in tokenized_text:
    seq = [word2idx[token] for token in tokens if token in word2idx]
    sequences.append(seq)

# Pad the sequences to a fixed length
padded_sequences = pad_sequence([torch.tensor(seq) for seq in sequences], batch_first=True, padding_value=0)

# Convert the sequences to PyTorch tensors
sequences_matrix = padded_sequences
print(sequences_matrix.shape)
print(X_train[:5])


y_train_tensor = torch.tensor(y_train.values)
y_test_tensor = torch.tensor(y_test.values)

# Print the shape of the tensors
print(y_train_tensor.shape)
print(y_test_tensor.shape)



In [ ]:
neg, pos = np.bincount(y_train_tensor)

# Calculate total samples.
total = neg + pos

# Calculate the weight for each label.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = torch.tensor([weight_for_0,weight_for_1 ]).float()

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class sms(nn.Module):
    def __init__(self, max_words, max_len):
        super(sms, self).__init__()
        self.embedding = nn.Embedding(max_words, 32)
        self.lstm = nn.LSTM(32, 64)
        self.dense1 = nn.Linear(64, 128)
        self.dropout = nn.Dropout(0.5)
        self.dense2 = nn.Linear(128, 2)
        #self.elu = nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        self.layer1=torch.nn.LayerNorm(128)
        self.layer2=torch.nn.LayerNorm(1)
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x, targets=None):
        x = self.embedding(x)
        x = self.dropout(x)
        output, (h, c) = self.lstm(x)
        output = output.squeeze(0)
        x = self.dense1(output)
        x=self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        logits = self.dense2(x)
        #logits = self.layer2(x)
        #logits = self.elu(x)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits=logits.mean(1)
            targets = targets.long()
            loss = F.cross_entropy(logits,targets,weight=class_weight)
        return logits, loss

# Example usage


# Create an instance of the model
model = sms(max_words, max_len)

# Perform a forward pass to compute the loss
outputs = model(sequences_matrix)


In [ ]:
learning_rate = 1e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
block_size = 32
eval_iters = 200
batch_size = 64
eval_interval=1000
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = sequences_matrix
    ix = torch.randint(len(data), (batch_size,))
    x = torch.stack([data[i] for i in ix])
    y = torch.stack([y_train_tensor[i] for i in ix])
    return x, y

for iter in range(10000):

    # every once in a while evaluate the loss on train and val sets
    #if iter % eval_interval == 0 or iter == max_iters - 1:
     #   losses = estimate_loss()
      #  print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits,loss= model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if iter % eval_interval == 0 or iter == 100000 - 1:
      logits,loss= model(sequences_matrix,y_train_tensor)
      print(f"step {iter}: train loss {loss:.4f}",logits[:10],y_train_tensor[:10])



In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_train = [pred_text]
    tokenized_text = [tokenizer(text) for text in pred_train]
    sequences = []
    for tokens in tokenized_text:
        seq = [word2idx[token] for token in tokens if token in word2idx]
        sequences.append(seq)
    padded_sequences = pad_sequence([torch.tensor(seq) for seq in sequences], batch_first=True, padding_value=0)
    logits = model(padded_sequences)
    if isinstance(logits, tuple):
        logits = logits[0]  # Assuming the model returns a tuple of (logits, ...)
    logits = torch.tensor(logits)
    counts = torch.exp(logits)
    probs = counts / counts.sum(1, keepdims=True)

    return (probs[0][0], ("ham" if probs[0][0] < 0.5 else "spam"))
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)